![](img/572_banner.png)

# Lab 1: Floating Point Numbers, Optimization & Gradient Descent

**Tomas Beuzen, January 2021**

In this lab, we'll work on solidifying concepts learned in Lecture 1 (floating point numbers) and Lecture 2 (optimization & gradient descent). This is the "vegetables" lab of the course - you're probably eager to get to neural networks, but we need to practice these fundamentals first! This lab contains quite a few pretty difficult optional exercises for those looking to go the extra mile - only do them if you have time! Good luck!

![](img/vegetable.png)

## Table of Contents
<hr>

<div class="toc"><ul class="toc-item"><li><span><a href="#Instructions" data-toc-modified-id="Instructions-2">Instructions</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-3">Imports</a></span></li><li><span><a href="#Exercise-1:-Floating-Point-Numbers" data-toc-modified-id="Exercise-1:-Floating-Point-Numbers-4">Exercise 1: Floating Point Numbers</a></span></li><li><span><a href="#Exercise-2:-logpdf" data-toc-modified-id="Exercise-2:-logpdf-5">Exercise 2: logpdf</a></span></li><li><span><a href="#Exercise-3:-Gradient-Descent-Learning-Rate" data-toc-modified-id="Exercise-3:-Gradient-Descent-Learning-Rate-6">Exercise 3: Gradient Descent Learning Rate</a></span></li><li><span><a href="#Exercise-4:-Logistic-Regression-from-Scratch" data-toc-modified-id="Exercise-4:-Logistic-Regression-from-Scratch-7">Exercise 4: Logistic Regression from Scratch</a></span></li><li><span><a href="#(Optional)-Exercise-5:-log(1-+-exp(z))" data-toc-modified-id="(Optional)-Exercise-5:-log(1-+-exp(z))-8">(Optional) Exercise 5: <code>log(1 + exp(z))</code></a></span></li><li><span><a href="#(Optional)-Exercise-6:-softmax-logistic-regression-and-log-sum-exp" data-toc-modified-id="(Optional)-Exercise-6:-softmax-logistic-regression-and-log-sum-exp-9">(Optional) Exercise 6: softmax logistic regression and log-sum-exp</a></span></li><li><span><a href="#Submit-to-Canvas-and-GitHub" data-toc-modified-id="Submit-to-Canvas-and-GitHub-10">Submit to Canvas and GitHub</a></span></li></ul></div>

## Instructions
<hr>

rubric={mechanics:3}

**Link to your GitHub repository:**

You will receive marks for correctly submitting this assignment. To submit this assignment you should:

1. Push your assignment to your GitHub repository!
2. Provide a link to your repository in the space provided above.
2. Upload a HTML render of your assignment to Canvas. The last cell of this notebook will help you do that.
3. Be sure to follow the [General Lab Instructions](https://ubc-mds.github.io/resources_pages/general_lab_instructions/). You can view a description of the different rubrics used for grading in MDS [here](https://github.com/UBC-MDS/public/tree/master/rubric).

Here's a break down of the required and optional exercises in this lab:

|         | Number of Exercises | Points |
|:-------:|:-------------------:|:------:|
| Required| 20 | 40 |
| Optional| 4  | 4 |

## Imports
<hr>

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats
from scipy.optimize import minimize
import plotly.graph_objects as go
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, log_loss
from utils.floating_point import *
from canvasutils.submit import submit, convert_notebook

## Exercise 1: Floating Point Numbers
<hr>

In this exercise you're going to explore floating point behaviour. You'll be provided with a code snippet which you should run and then you need to explain the result. Below are three example snippets and answers:

**Sample Code Snippet 1**

In [ ]:
0.3 - 0.2 - 0.1

**Sample Answer**: The result is not zero because 0.3, 0.2, and 0.1 are not represented exactly as floating point numbers.

**Sample Code Snippet 2**

In [ ]:
0.5 - 0.25 - 0.125 - 0.125

**Sample Answer**: The result is zero because 0.5 ($2^{-1}$), 0.25 ($2^{-2}$), and 0.125 ($2^{-3}$) are powers of 2 and there they _are_ represented exactly as floating point numbers. There is no roundoff error present. 

**Sample Code Snippet 3**

In [ ]:
0.4 - 0.2 - 0.2

**Sample Answer**: The result is correct (zero) despite the fact that 0.4 and 0.2 are not represented exactly as floating point numbers. This is a case of good luck: while 0.4 and 0.2 are not represented exactly, the roundoff errors happened to cancel out during the subtractions.

**Hint:** You can determine if a number can be exactly represented using a function I made called `float_rep()` from `utils.floating_point` that we imported at the top of the notebook. For example:

In [ ]:
float_rep(0.25)

In [ ]:
float_rep(0.2)

In [ ]:
float_rep(0.4)

### 1.1
rubric={reasoning:1}

In [ ]:
30 - 20 - 10

Your solution goes here.


### 1.2
rubric={reasoning:1}

In [ ]:
30.0 - 20.0 - 10.0

Your solution goes here.


### 1.3
rubric={reasoning:1}

In [ ]:
(10.0 ** 100 + 1) == 10.0 ** 100

Your solution goes here.


### 1.4
rubric={reasoning:1}

In [ ]:
(10.0 ** 100000 + 1) - 10.0 ** 100000

Your solution goes here.


### 1.5
rubric={reasoning:1}

In [ ]:
np.exp(1000) - np.exp(1000)

Your solution goes here.


### 1.6
rubric={reasoning:1}

In [ ]:
1 / np.exp(1000) == np.exp(-1000)

Your solution goes here.


### 1.7
rubric={reasoning:1}

In [ ]:
np.exp(1000) == np.exp(10000)

Your solution goes here.


### 1.8
rubric={reasoning:1}

In [ ]:
x = np.ones(100000)
x[0] = 1e20

y = np.ones(100000)
y[-1] = 1e20

sum(x) == sum(y)

Your solution goes here.


## Exercise 2: logpdf
<hr>

### 2.1
rubric={reasoning:2}

In most scientific computing libraries there are functions to compute the probability density function (pdf) of a probability distribution, and separate functions to compute the log of the pdf. For example, looking at the [R normal distribution documentation](https://stat.ethz.ch/R-manual/R-devel/library/stats/html/Normal.html), you will see that you can set `log=TRUE` or `log=FALSE`. Likewise in Python we have `scipy.stats.norm.pdf` and `scipy.stats.norm.logpdf`.

You might wonder why we bother creating these extra functions, when we can just take the log ourselves. For example, why do we need this: 

In [ ]:
scipy.stats.norm.logpdf(1)  # is this function useless?

When we can do this:

In [ ]:
np.log(scipy.stats.norm.pdf(1))  # seems to do the same thing as above?

In the space below, tell me why do we need these "extra" functions for computing the log of a pdf?

>Hint: try using a larger number in the functions above and see what happens and if you can explain the result...

Your solution goes here.



### (Optional) 2.2
rubric={accuracy:1}

Write a numerically safe function `log_gaussian_pdf` that computes the log of the standard Gaussian PDF (zero mean, unit variance). Your function should produce the same result as `scipy.stats.norm.logpdf` when evaluated at $x=100$.

> Hint: try working out the log of the [Gaussian PDF](https://en.wikipedia.org/wiki/Normal_distribution) by hand and coding that up directly. Sometimes re-arranging a formula directly can help avoid overflow errors! (Yay for math + CS + programming)!

In [ ]:
def log_gaussian_pdf(x):
    pass  # Your code here

assert log_gaussian_pdf(100) == scipy.stats.norm.logpdf(100), "Result not the same as scipy.stats.norm.logpdf"
print("Passed!")

Your solution goes here.



## Exercise 3: Gradient Descent Learning Rate
<hr>

Below are the same functions we saw in Lecture 2 for calculating the MSE and gradient of the MSE for a linear regression model $\hat{y}=\boldsymbol{w^T}\boldsymbol{x}$, and a function for doing gradient descent. Note that the gradient descent function returns both the final optimized `w` and a list of the loss at each iteration (you'll need this "loss history" for a leter question).

In [ ]:
def mse(w, X, y):
    """Mean squared error.
    
    Parameters
    ----------
    w : ndarray
        Weight vector.
    X : ndarray
        Feature data.
    y : ndarray
        Response data.
    
    Returns
    -------
    ndarray
        Mean squared error.
    """
    return np.mean((X @ w - y) ** 2)


def mse_grad(w, X, y):
    """Gradient of mean squared error.
        
    Parameters
    ----------
    w : ndarray
        Weight vector.
    X : ndarray
        Feature data.
    y : ndarray
        Response data.
    
    Returns
    -------
    ndarray
        Gradient of MSE loss w.r.t parameters w.
    """
    return X.T @ (X @ w) - X.T @ y


def gradient_descent(f, f_grad, w, X, y, alpha, ϵ=2e-4, max_iterations=500, verbose=False):
    """Gradient descent.
    
        
    Parameters
    ----------
    f : ndarray
        Function to minimize.
    f_grad : ndarray
        Gradient of function w.r.t weights.
    w : ndarray
        Weight vector.
    X : ndarray
        Feature data.
    y : ndarray
        Response data.
    alpha : float
        Learning rate.
    ϵ : float
        Stop GD if step-size smaller than this.
    max_iterations : int
        Maximum iterations of GD to do.
    verbose : bool
        Whether to print progress.
    
    Returns
    -------
    ndarray
        Mean squared error.
    """

    iterations = 1         # init. iterations
    dw = np.array(2 * ϵ)   # init. dw
    losses = [f(w, X, y)]  # init. loss history 
    while abs(dw).sum() > ϵ and iterations <= max_iterations:
        g = f_grad(w, X, y)  # calculate current gradient
        dw = alpha * g       # change in w
        w = w - dw           # adjust w based on gradient * learning rate
        iterations += 1      # increase iteration
        losses.append(f(w, X, y))
        if verbose and iterations % 50 == 0: print(f"Loss: {f(w, X, y):.4f}")
    if verbose: print(f"Terminated after {iterations - 1} iterations!")
    return w, losses

Let's create a simple regression dataset to test our gradient descent function. Our model is going to be a simple linear regression model, $\hat{y_i}=wx_i$, with just one parameter to fit, $w$, and no intercept.

In [ ]:
X, y = make_regression(100, 1, random_state=2020)

Let's fit our simple linear model $\hat{y_i}=wx_i$:

In [ ]:
np.random.seed(123)  # fix the random seed for reproducibility
w0 = np.random.randint(0, 20, 1)  # random intial weight
w, losses = gradient_descent(mse, mse_grad, w0, X, y, alpha=0.001, verbose=True)
print(f"Initial w = {w0[0]:.2f}")
print(f"Optimum w = {w[0]:.2f}")

To confirm everything is working as expected, let's compare our result to scikit learn's `LinearRegression`:

In [ ]:
lr = LinearRegression(fit_intercept=False).fit(X, y)
print(f"w = {lr.coef_[0]:.2f}")

Looks good!

> You could of course fit a more complex model if you wanted to. For example, if you wanted to fit a model with 3 parameters, we would just re-create our regression dataset to have 3 features and off we go:

```python
X, y = make_regression(100, 3, random_state=2020)
w0 = np.random.randint(0, 20, 3)  # random intial weight
w, losses = gradient_descent(mse, mse_grad, w0, X, y, alpha=0.001, verbose=True)
for n, _ in enumerate(w):
    print(f"w{n} = {_:.2f}")
lr = LinearRegression(fit_intercept=False).fit(X, y)
print("sklearn coefs:", lr.coef_)
```

### 3.1
rubric={viz:4}

As we saw in Lecture 2, in the case of linear regression, the MSE loss is [convex](https://en.wikipedia.org/wiki/Convex_function), that basically means that there is a "global minimum" that gradient descent will always be able to find (if we do enough iterations and the learning rate is not too high that the gradients explode). This is what the "loss surface" looks like - i.e., the loss function for different values of $w$:

>If the below doesn't run, you may need the plotly extension. Run this from your base environment: `jupyter labextension install jupyterlab-plotly@4.14.2`.

In [ ]:
# Don't worry too much about this plotting code, it's just for illustration purposes
weights = np.arange(0, 41, 1)
loss = [mse(np.array([_]), X, y) for _ in weights]
w, losses = gradient_descent(mse, mse_grad, w0, X, y, alpha=0.001, verbose=False)
fig = go.Figure()
fig.add_trace(go.Scatter(x=weights, y=loss, name="Loss"))
fig.add_trace(go.Scatter(x=w0, y=losses[:1], mode="markers", marker=dict(size=14, line=dict(width=1, color="DarkSlateGrey")), name="Initial w"))
fig.add_trace(go.Scatter(x=w, y=losses[-1:], mode="markers", marker=dict(size=14, line=dict(width=1, color="DarkSlateGrey")), name="Optimal w"))
fig.update_layout(width=600, height=400, margin=dict(t=30)); fig.update_xaxes(title="w", dtick=2); fig.update_yaxes(title="MSE loss")

See how it's convex? Remember with gradient descent we are iteratively taking steps "down hill" so it makes sense that with each iteration of gradient descent (i.e., each "step" down hill), our loss *should* go down and move us towards that "global minimum" (assuming our learning rate is not too high).

Using the gradient descent code and data we defined at the beginning of the exercise, I want you to show that loss decreases with iterations by making a plot of `loss` vs `iterations` for the following five learning rates: 0.00001, 0.00005, 0.0001, 0.001, 0.022. Your plot should look something like the below and have 5 lines on it (**you don't have to use plotly though, use any plotting library you like!**):

![](img/plot.png)

**Hint:** I've coded up our function above so that it returns the losses. For example, to get the losses per iteration for `α=0.00001`, run: `w, losses = gradient_descent(mse, mse_grad, w0, X, y, alpha=0.00001)` and you can then plot `losses`. It doesn't matter what `w0` you use for this exercise.

In [ ]:
# Your solution here.

### 3.2
rubric={reasoning:2}

If all went well, you probably saw in the plot above that as learning rate increases, the number of iterations to convergence typically decreases - makes sense, a higher learning rate means we are taking bigger steps towards the minimum. **However** the final learning rate of 0.022 actually converged slower than a rate of 0.001. Just in case your plot didn't show this, we can confirm by running our code again:

In [ ]:
gradient_descent(mse, mse_grad, w0, X, y, alpha=0.001, verbose=True);

In [ ]:
gradient_descent(mse, mse_grad, w0, X, y, alpha=0.022, verbose=True);

Why does a learning rate of 0.022 converge slower here?

Your solution goes here.



### 3.3
rubric={reasoning:2}

Based on the data and simple linear regression model we are using in this exercise, answer the following:

1. Does the number of iterations it takes gradient descent to converge depend on the initial weight (`w0`) we specify?
2. Does gradient descent always converge to the same optimum `w` regardless of the initial weight (`w0`) value?

Your solution goes here.



### 3.4
rubric={reasoning:2}

In the next few lectures we'll get to neural networks which are typically non-linear models. This makes them harder to optimize, especially with gradient descent! Non-linearities in our model lead to loss surfaces that are no longer convex. They are bumpy and have what we call "local minima" which we can get stuck in (there are also things called "saddle points" where we can get stuck but we'll talk about those in a later lecture). Gradient descent is particularly prone to getting stuck in local minimum. We'll explore what that means in this exercise.

Imagine we now want to fit a non-linear model, $\hat{y_i} = wcos(wx_i)$, to our dataset (rather than our previous linear model $\hat{y_i}=wx_i$). I've coded up that function (`nonlin`), the mse loss function (`mse_nonlin`), and the gradient of the loss function  (`mse_nonlin_grad`) below, along with a plotting function (I'm omitting docstrings now to be concise). Run the cell.

In [ ]:
X, y = make_regression(100, 1, random_state=2020)
X = X.flatten()

def nonlin(w, x):
    return w * np.cos(w * x)

def mse_nonlinear(w, x, y):
    """Mean squared error."""
    return np.mean((nonlin(w, x) - y) ** 2)

def mse_nonlinear_grad(w, x, y):
    """Gradient of mean squared error."""
    t = np.cos(w * x) - w * x * np.sin(w * x)
    return np.mean((nonlin(w, x) - y) * t)

def plot_loss_vs_weight(w0, alpha=0.01):
    """Plotting function using plotly - don't worry too much about the code here!"""
    if abs(w0) > 10: raise ValueError("Please specify w0 between -10 and 10")
    wf = gradient_descent(mse_nonlinear, mse_nonlinear_grad, w0, X, y, alpha=alpha, ϵ=2e-5, verbose=False)[0];
    weights = np.arange(-10, 10.1, 0.1)
    loss = [mse_nonlinear(_, X, y) for _ in weights]
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=weights, y=loss, name="Loss"))
    fig.add_trace(go.Scatter(x=[w0], y=[mse_nonlinear(w0, X, y)], mode="markers", marker=dict(size=14, line=dict(width=1, color="DarkSlateGrey")), name="Initial W"))
    fig.add_trace(go.Scatter(x=[wf], y=[mse_nonlinear(wf, X, y)], mode="markers", marker=dict(size=14, line=dict(width=1, color="DarkSlateGrey")), name="Final W"))
    fig.update_layout(width=600, height=500)
    fig.update_xaxes(title="w", dtick=2)
    fig.update_yaxes(title="loss")
    return fig

Let's go ahead and plot this loss surface for different weights. Our plotting function also takes in an initial weight, `w0` and uses gradient descent to find the optimal weight and plots those too. Run the cell below to see what I mean:

In [ ]:
w0 = 9
plot_loss_vs_weight(w0, alpha=0.01)

See how our non-linear model results in this non-linear loss surface? It's very different to our nice convex one from earlier! Play around with the inital weight `w0` and answer the following:

1. Does the number of iterations it takes gradient descent to converge depend on the initial weight (`w0`)?
2. Does gradient descent always converge to the same optimum `w` regardless of the initial weight (`w0`) value?

Your solution goes here.



### 3.5
rubric={reasoning:1}

In practice, you don't know the loss surface in advance (I've been able to plot it here because our model only has a single parameter and our dataset is very simple). In fact, when we get to neural networks, we'll be dealing with tens, hundreds, thousands, or even millions of parameters resulting in an incredibly complex loss surface that is impossible to visualize (imagine that plot above but in hundreds of dimensions 🤯 ). Without knowing what the loss surface looks like you won't know whether you've optimized your model based on the global minimum or if you're stuck in a local minimum. What is one method you could use to try and avoid getting stuck in a local minima?

Your solution goes here.



## Exercise 4: Logistic Regression from Scratch
<hr>

In this exercise, I'm going to get you to implement your very own Logistic Regression model from scratch - just like sklearn's `LogisticRegression` class. This is going to be so cool - I hope after this exercise you have a much better understanding of how sklearn models work, you could even start building your own ML library, the world is your oyster!

To train and test our model, we're going to try predicting the presence of [Pulsar Stars](https://en.wikipedia.org/wiki/Pulsar), using a dataset available on the UCI Machine Learning Repository [here](https://archive.ics.uci.edu/ml/datasets/HTRU2). Understanding the dataset is not that important to the question but I'll briefly explain it below.

Pulsar stars are a rare type of (dead) star that produce regular pulses of radiation in two symmetrical beams, like a lighthouse, as shown in the gif below. They can rotate up to hunderds of times per second!

![](https://www.ligo.org/science/Publication-S6VSR24KnownPulsar/Images/lightnew.gif)

Source: [www.ligo.org](https://www.ligo.org/science/Publication-S6VSR24KnownPulsar/Images/lightnew.gif)

Pulsars are hard to find, yet they are extremely useful for a variety of astrophysical purposes. As pulsars spin, their radiation beams sweep across the sky and can be detected from Earth (if they cross our line of sight), producing a detectable pattern of radiation in our measurment devices. Scientists typically average this signal over many rotations of the pulsar and then describe the pattern using various statical measures like the mean, standard deviation, skewness, etc. These are the features we have access to in this exercise. We have a training set of 4 features and 9273 observations, and we wish to classify which observations are pulsar stars (i.e., we have a simple binary classification problem). You can read more about pulsar stars and this dataset [here](https://archive.ics.uci.edu/ml/datasets/HTRU2).

The code below reads in the data:

In [ ]:
df = pd.read_csv("data/pulsar_data.csv")
df.head()

### 4.1
rubric={accuracy:1}

First, use sklearn's `train_test_split()` to split data into training and testing sets using the `test_size` and `random_state` given below.

In [ ]:
test_size = 0.2
random_state = 2021

# Your solution goes here

### 4.2
rubric={accuracy:2}

Now, standardise the feature data in the training and test sets using sklearn's `StandardScaler()`. As we saw in Lecture 2, standardising often helps with optimization.

In [ ]:
# Your solution goes here

### 4.3
rubric={accuracy:6}

Okay, time to build our very own logistic regression class! The cell below defines the logistic loss function and its gradient which we saw previously in [Lecture 2](https://pages.github.ubc.ca/MDS-2020-21/DSCI_572_sup-learn-2_students/lectures/lecture2_gradient-descent.html#lecture-exercise-optimizing-logistic-regression) (you can also find their derivations in [Appendix B](https://pages.github.ubc.ca/MDS-2020-21/DSCI_572_sup-learn-2_students/lectures/appendixB_logistic-loss.html)). Run the cell - you'll need these functions.

In [ ]:
def sigmoid(w, X):
    """Sigmoid function."""
    return 1 / (1 + np.exp(-X @ w))

def logistic_loss(w, X, y):
    """Logistic loss."""
    return -(y * np.log(sigmoid(w, X)) + (1 - y) * np.log(1 - sigmoid(w, X))).mean()

def logistic_loss_grad(w, X, y):
    """Gradient of logistic loss."""
    return (X.T @ (sigmoid(w, X) - y) / len(X))

Now, complete the code below to create the class `MyLogisticRegression` - this is our custom logistic regression class with its very own `fit`, `predict`, `predict_proba`, and `score` methods. Here is what you should do:

1. Use `scipy.optimize.minimize` in the `fit` method to solve the optimization problem (we did this in [Lecture 2 here](https://pages.github.ubc.ca/MDS-2020-21/DSCI_572_sup-learn-2_students/lectures/lecture2_gradient-descent.html#lecture-exercise-optimizing-logistic-regression)). I suggest initializing your weights to all zeros in the `fit` method before calling `minimize()` and passing in the gradient (`logistic_loss_grad`) to `minimize()` using the `jac` argument, just like we did in Lecture 2.
2. `predict_proba` should just return the predicted probabilities from your model (i.e., the output of the `sigmoid()` function). I've done this for you.
3. `predict` should convert predicted probabilities to 0's and 1's depending on the `threshold`.
4. `score` should return the accuracy of the model, i.e., the accuracy when comparing the output of `predict` to the true class values `y`.
5. I've provided some `assert` statements to help you see if you're on the right track.

This is so cool, I love doing things from scratch and opening up the "black-box", I'm getting all giddy writing this question. If you can do this question, I have full faith that you have a good understanding of logistic regression, and of optimization.

In [ ]:
class MyLogisticRegression:
    
    def __init__(self, method="L-BFGS-B"):
        self.method = method  # optimization method to use in scipy.optimize.minimize

    def fit(self, X, y):
        """Fit the model according to the given training data."""
        pass  # Your solution goes here

    def predict_proba(self, X):
        """Probability estimates for samples in X."""
        return sigmoid(self.w, X)
    
    def predict(self, X, threshold=0.5):
        """Predict class labels (0 or 1) for samples in X."""
        pass  # Your solution goes here
    
    def score(self, X, y, threshold=0.5):
        """Accuracy of model on the given test data and labels."""
        pass  # Your solution goes here
    


In [ ]:
model = MyLogisticRegression()
model.fit(X_train, y_train)
assert model.w.size == 4, "Wrong number of parameters"
assert (model.predict_proba(X_test) <= 1).all(), "Predicted probabilities exceed 1"
assert set(model.predict(X_test)) == {0, 1}, "Model predicts value other than 0 and 1"
assert np.isclose(model.score(X_train, y_train), 0.93, atol=0.05), "Model score on training data seems to be off..."
assert np.isclose(model.score(X_test, y_test), 0.92, atol=0.05), "Model score on training data seems to be off..."

> Do you get a `divide by zero encountered in log` when running the code above and fitting your model? AWESOME! This is caused by underflow, overflow and floating point rounding errors. If you're interested, read on, if you're not, ignore me. The error occurs when we try to take the log of the sigmoid function in the `logistic_loss()` function: `np.log(sigmoid(w, X))` or `np.log(1 - sigmoid(w, X))`. The term `sigmoid(w, X)` may be 0 or 1 if `np.exp(-X @ w)` in the `sigmoid()` function overflows or underflows respectively, and that means we'd be trying to take the log of 0 which gives us the error. If you're interested in learning more, try out the optional exercises 5 and 6 to come up with clever ways of avoiding this error - the tricks you'll implement are used in real Python computing packages! In fact, if you replace the `logistic_loss()` function I defined earlier with the sklearn implementation, you'll avoid this error because sklearn uses the tricks in optional exercises 5 and 6 to avoid underflow/overflow errors:

```python
from sklearn.metrics import log_loss
def logistic_loss(w, X, y):
    y_hat = sigmoid(w, X)
    return log_loss(y, y_hat)
```

>SOOOO COOOOOOOL!!!!!! I love it!

### 4.4
rubric={accuracy:3}

Report the training accuracy and test accuracy of your model along with a confusion matrix for the test data.

In [ ]:
# Your solution goes here.

### 4.5
rubric={accuracy:2, reasoning:2}

Compare the fitting speed, fitted coefficients and test accuracy of your logistic regression implementation called `MyLogisticRegression` to the sklearn one (`from sklearn.linear_model import LogisticRegression`). For a fair comparison, use the following hyperparameters with scikit-learn's `LogisticRegression`:

- `C=1e8` to (mostly) disable regularization for sklearn, since your implementation doesn't use regularization. 
- `fit_intercept=False` since your code above doesn't fit an intercept term.

Is there a significant difference in the fit speed, coefficients or test accuracy between the two models?

In [ ]:
# Your solution goes here.

### (Optional) 4.6
rubric={reasoning:1}

This question is trickier, I'm getting a little carried away but like I said, I love doing things from scratch so let's throw in some regularization - yewwwwwww!. By default, sklearn's `LogisticRegression` implements regularization - which is typically a good idea. Modify our custom class `MyLogisticRegression` to include L2 regularization. You actually don't need to change the `MyLogisticRegression` class at all for this, we just need to change the loss function and gradient of the loss function!

Here's the log loss function we're already familiar with:

$$f(w)=-\frac{1}{n}\sum_{i=1}^ny_i\log\left(\frac{1}{1 + \exp(-w^Tx_i)}\right) + (1 - y_i)\log\left(1 - \frac{1}{1 + \exp(-w^Tx_i)}\right)$$

And here it is with L2 regularization:

$$f(w)=-\frac{1}{n}\sum_{i=1}^ny_i\log\left(\frac{1}{1 + \exp(-w^Tx_i)}\right) + (1 - y_i)\log\left(1 - \frac{1}{1 + \exp(-w^Tx_i)}\right) + \frac{\lambda}{2n}\sum_{j=1}^dw_j^2$$

Where $d$ is the number of parameters in the model (4 in this case, one for each feature), and $\lambda$ is the regularization parameter. Re-define the functions `logistic_loss()` and `logistic_loss_grad()` to include L2 regularization (you'll need to work out the formular for the gradient of the loss yourself, it's not very difficult). Your model should then give similar results to sklearn's `LogisticRegression()` with the parameter `C=1` - I've included a cell for you to test this out.

In [ ]:
def logistic_loss(w, X, y, λ=1):
    pass # your answer goes here

def logistic_loss_grad(w, X, y, λ=1):
    pass # your answer goes here


In [ ]:
# Test that model coefficients are similar to sklearn
my_model = MyLogisticRegression()
my_model.fit(X_train, y_train)
sk_model = LogisticRegression(C=1, fit_intercept=False)
sk_model.fit(X_train, y_train)

print(f"MyLogisticRegression coefs: {my_model.w}")
print(f"sklearn LogisticRegression coefs: {sk_model.coef_[0]}")

## (Optional) Exercise 5: `log(1 + exp(z))`
<hr>

rubric={accuracy:1}

[In Lecture 1 we discussed computing](https://pages.github.ubc.ca/MDS-2020-21/DSCI_572_sup-learn-2_students/lectures/lecture1_floating-point.html#writing-functions) $\log(1+\exp(z))$ and we wrote a better version of the function for when $z\gg1$ which avoids overflow errors:

In [ ]:
def log_1_plus_exp(z):
    return np.log(1 + np.exp(z))


def log_1_plus_exp_safe(z):
    if z > 100:
        return z
    else:
        return np.log(1 + np.exp(z))

You might have thought this was not all that important at the time, but as we just saw in Exercise 4.3, even fairly simple equations like the log loss function are at the mercy of underflow and overflow errors. Now let's consider the case of $z\ll -1$, i.e., when $z$ is a large negative number. In that case, we get zero with both of the above implementations:

In [ ]:
print(log_1_plus_exp(-100))

In [ ]:
print(log_1_plus_exp_safe(-100))

Your tasks:

1. Investigate why this is happening. Is the problem that $\exp(-100)$ itself underflows?
2. Write a function `log_1_plus_exp_safer` that works well when $z\gg 1$ and $z\ll -1$.
3. For what range of values of $z$ does your `log_1_plus_exp_safer` function give reasonable results?

Your solution goes here.



## (Optional) Exercise 6: softmax logistic regression and log-sum-exp
rubric={reasoning:1}

In the "multinomial" (aka softmax) approach to multi-class logistic regression, your loss ends up having one term for each class, so you get something of the form $\log \sum_{k=1}^c \exp(z_k)$, where $c$ is the number of classes, and $z_k$ is a quantity involving the weights and training data (we'll talk more about softmax later in the course, you don't really need to be familiar with it to answer this question - it's all math and floating point here). For any choice of a constant $a$, we can rewrite this as follows:

$$\begin{align}\log \displaystyle\sum_{k=1}^c \exp(z_k) &= \log \displaystyle\sum_{k=1}^c \exp(z_k-a+a)\\ &= \log \left( \displaystyle\sum_{k=1}^c \exp(z_k-a)\exp(a) \right) \\ &= \log \left( \exp(a)\displaystyle\sum_{k=1}^c \exp(z_k-a)\right) \\ &= \log \left( \exp(a) \right) + \log\displaystyle\sum_{k=1}^c \exp(z_k-a) \\ &= a+ \log \displaystyle\sum_{k=1}^c \exp(z_k-a)\end{align}$$

Note: you only need to look at the first and last expression - the middle parts are only there to convince you they are indeed equivalent.

1. Explain why this final expression might be more numerically stable and why $a=\max \{z_1,z_2,\ldots,z_c\}$ is a sensible choice.
2. If $a=\max \{z_1,z_2,\ldots,z_c\}$, this trick seems to rely on the fact that overflow is more of a danger than underflow, because we may now compute $exp$ of some very large negative values if $a$ is large. Explain why overflow is more of a problem than underflow in this calculation.
3. Write a python function `log_sum_exp` that takes in an array `z` and computes the _original_ expression. Write another function `log_sum_exp_stable` that computes the final (more stable) expression using $a=\max \{z_1,z_2,\ldots,z_c\}$. Discuss the behaviour of the two functions. Also, compare your implementation to In fact, [`scipy.special.logsumexp`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.logsumexp.html#scipy.special.logsumexp) for one or two cases.
4. Earlier, we used the approximation $\log(1+\exp(z))\approx z$ when $z\gg 1$. Is that just a specific case of what we have here? It seems that what we did earlier was an approximation, whereas what we did here is mathematically exact. Do you think there is any significance to this distinction, in practice?

>FYI: you see this trick in real implementations of ML algorithms all the time!

Your solution goes here.



## Submit to Canvas and GitHub
<hr>

When you are ready to submit your assignment do the following:
1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Run All Cells...`
2. Save your notebook.
3. Convert your notebook to `.html` format using the `convert_notebook()` function below or by `File -> Export Notebook As... -> Export Notebook to HTML`
4. Run the code `submit()` below to go through an interactive submission process to Canvas.
5. Finally, push all your work to GitHub (including the rendered html file).

In [ ]:
# convert_notebook("lab1.ipynb", "html")  # save your notebook, then uncomment and run when you want to convert to html

In [ ]:
# submit(course_code=59090)  # uncomment and run when ready to submit to Canvas

![](img/mood.png)